In [255]:
import pandas as pd
import numpy as np
import requests
import ast

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. 
El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.
- Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

In [256]:
df_attacks = pd.read_csv('../../datos/attacks_limpio_5.csv', index_col=0)
df_meteo = pd.read_csv('../../datos/paises_meteo.csv', index_col=0)

In [257]:
df_attacks.head(2)

,case_number,mes,country,type,area,activity,sex,species,injury,fatal,year,age
0,1800.00.00,NaN,seychelles,Unprovoked,St. Anne,a corsair's boat was overturned,F,NaN,"FATAL, all onboard were killed by sharks",y,1976.0,26.515198
1,1797.05.28.R,May,NaN,Unprovoked,NaN,Dropped overboard,M,NaN,FATAL,y,1976.0,26.515198


In [258]:
df_meteo.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 215, 'speed':...",25,2,6,1013,none,0,0,130,2,39.78373,-100.445882,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 9}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 135, 'speed':...",24,2,5,1012,none,0,0,105,3,39.78373,-100.445882,USA


In [259]:
df_meteo_copia = df_meteo.copy()

In [260]:
df_attacks_paises = df_attacks[df_attacks['country'].isin(['australia', 'usa', 'new zealand', 'south africa', 'papua new guinea'])]

In [261]:
df_attacks_paises['country'].unique()

array(['australia', 'usa', 'papua new guinea', 'new zealand',
       'south africa'], dtype=object)

In [262]:
# Vemos que nuestra columna 'wind profile' es un string con forma de lista de diccionarios
df_meteo['wind_profile'][0]


"[{'layer': '950mb', 'direction': 215, 'speed': 2}, {'layer': '900mb', 'direction': 230, 'speed': 3}, {'layer': '850mb', 'direction': 255, 'speed': 3}, {'layer': '800mb', 'direction': 270, 'speed': 3}, {'layer': '750mb', 'direction': 275, 'speed': 3}, {'layer': '700mb', 'direction': 280, 'speed': 2}, {'layer': '650mb', 'direction': 330, 'speed': 2}, {'layer': '600mb', 'direction': 345, 'speed': 2}, {'layer': '550mb', 'direction': 325, 'speed': 3}, {'layer': '500mb', 'direction': 305, 'speed': 3}, {'layer': '450mb', 'direction': 290, 'speed': 4}, {'layer': '400mb', 'direction': 290, 'speed': 5}, {'layer': '350mb', 'direction': 290, 'speed': 5}, {'layer': '300mb', 'direction': 300, 'speed': 5}, {'layer': '250mb', 'direction': 315, 'speed': 6}, {'layer': '200mb', 'direction': 295, 'speed': 6}]"

In [263]:
# Usamos la librería ast para convertir nuestro string en la estructura que posee dentro
df_meteo['wind_profile']= df_meteo['wind_profile'].apply(ast.literal_eval)

In [264]:
# Vemos que ahora nuestra columna 'wind profile' es una lista de diccionarios y podemos pasar a limpiarla
df_meteo['wind_profile'][0]


[{'layer': '950mb', 'direction': 215, 'speed': 2},
 {'layer': '900mb', 'direction': 230, 'speed': 3},
 {'layer': '850mb', 'direction': 255, 'speed': 3},
 {'layer': '800mb', 'direction': 270, 'speed': 3},
 {'layer': '750mb', 'direction': 275, 'speed': 3},
 {'layer': '700mb', 'direction': 280, 'speed': 2},
 {'layer': '650mb', 'direction': 330, 'speed': 2},
 {'layer': '600mb', 'direction': 345, 'speed': 2},
 {'layer': '550mb', 'direction': 325, 'speed': 3},
 {'layer': '500mb', 'direction': 305, 'speed': 3},
 {'layer': '450mb', 'direction': 290, 'speed': 4},
 {'layer': '400mb', 'direction': 290, 'speed': 5},
 {'layer': '350mb', 'direction': 290, 'speed': 5},
 {'layer': '300mb', 'direction': 300, 'speed': 5},
 {'layer': '250mb', 'direction': 315, 'speed': 6},
 {'layer': '200mb', 'direction': 295, 'speed': 6}]

In [265]:
len(df_meteo['wind_profile'][0])

16

In [266]:
# Con el método pd.Series convertimos cada uno de los diccionarios 
# dentro de nuestra lista en una columna. 
x = df_meteo['wind_profile'].apply(pd.Series)

In [267]:
# Vemos que se nos crearon 16 columnas (la cantidad de diccionarios de nuestra lista)
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 215, 'speed': 2}","{'layer': '900mb', 'direction': 230, 'speed': 3}","{'layer': '850mb', 'direction': 255, 'speed': 3}","{'layer': '800mb', 'direction': 270, 'speed': 3}","{'layer': '750mb', 'direction': 275, 'speed': 3}","{'layer': '700mb', 'direction': 280, 'speed': 2}","{'layer': '650mb', 'direction': 330, 'speed': 2}","{'layer': '600mb', 'direction': 345, 'speed': 2}","{'layer': '550mb', 'direction': 325, 'speed': 3}","{'layer': '500mb', 'direction': 305, 'speed': 3}","{'layer': '450mb', 'direction': 290, 'speed': 4}","{'layer': '400mb', 'direction': 290, 'speed': 5}","{'layer': '350mb', 'direction': 290, 'speed': 5}","{'layer': '300mb', 'direction': 300, 'speed': 5}","{'layer': '250mb', 'direction': 315, 'speed': 6}","{'layer': '200mb', 'direction': 295, 'speed': 6}"
1,"{'layer': '950mb', 'direction': 135, 'speed': 3}","{'layer': '900mb', 'direction': 190, 'speed': 2}","{'layer': '850mb', 'direction': 270, 'speed': 3}","{'layer': '800mb', 'direction': 280, 'speed': 3}","{'layer': '750mb', 'direction': 270, 'speed': 3}","{'layer': '700mb', 'direction': 275, 'speed': 3}","{'layer': '650mb', 'direction': 310, 'speed': 2}","{'layer': '600mb', 'direction': 320, 'speed': 2}","{'layer': '550mb', 'direction': 305, 'speed': 2}","{'layer': '500mb', 'direction': 295, 'speed': 3}","{'layer': '450mb', 'direction': 295, 'speed': 4}","{'layer': '400mb', 'direction': 295, 'speed': 4}","{'layer': '350mb', 'direction': 300, 'speed': 5}","{'layer': '300mb', 'direction': 305, 'speed': 6}","{'layer': '250mb', 'direction': 310, 'speed': 6}","{'layer': '200mb', 'direction': 305, 'speed': 6}"
2,"{'layer': '950mb', 'direction': 130, 'speed': 2}","{'layer': '900mb', 'direction': 170, 'speed': 2}","{'layer': '850mb', 'direction': 235, 'speed': 2}","{'layer': '800mb', 'direction': 285, 'speed': 2}","{'layer': '750mb', 'direction': 305, 'speed': 3}","{'layer': '700mb', 'direction': 280, 'speed': 3}","{'layer': '650mb', 'direction': 295, 'speed': 2}","{'layer': '600mb', 'direction': 305, 'speed': 2}","{'layer': '550mb', 'direction': 300, 'speed': 3}","{'layer': '500mb', 'direction': 295, 'speed': 3}","{'layer': '450mb', 'direction': 295, 'speed': 3}","{'layer': '400mb', 'direction': 285, 'speed': 4}","{'layer': '350mb', 'direction': 295, 'speed': 5}","{'layer': '300mb', 'direction': 295, 'speed': 6}","{'layer': '250mb', 'direction': 305, 'speed': 6}","{'layer': '200mb', 'direction': 305, 'speed': 7}"
3,"{'layer': '950mb', 'direction': 85, 'speed': 2}","{'layer': '900mb', 'direction': 125, 'speed': 2}","{'layer': '850mb', 'direction': 195, 'speed': 2}","{'layer': '800mb', 'direction': 300, 'speed': 2}","{'layer': '750mb', 'direction': 300, 'speed': 3}","{'layer': '700mb', 'direction': 300, 'speed': 3}","{'layer': '650mb', 'direction': 315, 'speed': 2}","{'layer': '600mb', 'direction': 310, 'speed': 2}","{'layer': '550mb', 'direction': 290, 'speed': 2}","{'layer': '500mb', 'direction': 285, 'speed': 3}","{'layer': '450mb', 'direction': 280, 'speed': 3}","{'layer': '400mb', 'direction': 275, 'speed': 4}","{'layer': '350mb', 'direction': 280, 'speed': 5}","{'layer': '300mb', 'direction': 290, 'speed': 6}","{'layer': '250mb', 'direction': 300, 'speed': 6}","{'layer': '200mb', 'direction': 305, 'speed': 7}"
4,"{'layer': '950mb', 'direction': 30, 'speed': 2}","{'layer': '900mb', 'direction': 145, 'speed': 2}","{'layer': '850mb', 'direction': 150, 'speed': 2}","{'layer': '800mb', 'direction': 205, 'speed': 2}","{'layer': '750mb', 'direction': 285, 'speed': 3}","{'layer': '700mb', 'direction': 340, 'speed': 2}","{'layer': '650mb', 'direction': 60, 'speed': 2}","{'layer': '600mb', 'direction': 235, 'speed': 2}","{'layer': '550mb', 'direction': 250, 'speed': 2}","{'layer': '500mb', 'direction': 260, 'speed': 3}","{'layer': '450mb', 'direction': 270, 'speed': 3}","{'layer': '400mb', 'direction': 275, 'speed': 5}","{'layer': '350mb', 'direction': 280, 'speed': 5}","{'layer': '300mb', 'direct

In [268]:
# Crearemos una función para convertir mis diccionarios en columnas con la información buscada

def limpiar_columnas_meteo(df):

    df['wind_profile']= df['wind_profile'].apply(ast.literal_eval)

    x = df['wind_profile'].apply(pd.Series)

    for i in range(len(x.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
        nombre = 'windspeed_' + str(x[i].apply(pd.Series)['layer'][0])

        # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
        valores = list(x[i].apply(pd.Series)["speed"] )

        # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
        df.insert(i, nombre, valores)

    df['rh_profile']= df['rh_profile'].apply(ast.literal_eval)

    x = df['rh_profile'].apply(pd.Series)

    for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
        nombre = 'rh_' + str(x[i].apply(pd.Series)['layer'][0] )
        # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
        valores = list(x[i].apply(pd.Series)["rh"] )

        # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
        df.insert(i, nombre, valores)
    
    return df

In [269]:
df_meteo_copia = limpiar_columnas_meteo(df_meteo_copia)

In [270]:
df_meteo_copia

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,pais
0,9,13,11,9,7,4,3,3,4,3,...,6,1013,none,0,0,130,2,39.783730,-100.445882,USA
1,9,12,13,15,6,3,2,2,4,3,...,5,1012,none,0,0,105,3,39.783730,-100.445882,USA
2,9,12,13,12,9,3,1,2,2,2,...,6,1012,none,0,0,120,2,39.783730,-100.445882,USA
3,8,11,13,14,11,8,2,2,2,2,...,5,1013,none,0,0,80,2,39.783730,-100.445882,USA
4,9,11,12,14,12,8,2,2,2,2,...,6,1014,none,0,0,15,2,39.783730,-100.445882,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,10,12,11,-1,3,6,-1,-4,-4,-3,...,12,1012,none,3,0,100,4,-5.681607,144.248908,PAPUA NEW GUINEA
316,13,9,3,-1,4,5,-1,-4,-4,-3,...,11,1011,none,3,0,90,4,-5.681607,144.248908,PAPUA NEW GUINEA
317,14,10,1,0,4,4,-1,-4,-4,-3,...,11,1009,none,3,0,95,4,-5.681607,144.248908,PAPUA NEW GUINEA
318,14,13,6,0,4,4,-1,-4,-4,-4,...,12,1010,rain,3,0,95,4,-5.681607,144.248908,PAPUA NEW GUINEA


In [272]:
df_meteo_medias = df_meteo_copia.groupby('pais').apply(lambda x: x.mean())

/tmp/ipykernel_7380/422706665.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_meteo_medias = df_meteo_copia.groupby('pais').apply(lambda x: x.mean())


In [273]:
df_meteo_medias.reset_index(inplace=True)

In [274]:
df_meteo_medias['pais'] = df_meteo_medias['pais'].apply(lambda x: x.lower())

In [275]:
df_meteo_medias

,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,australia,13.343750,11.859375,7.062500,3.343750,1.171875,1.031250,0.46875,-0.375000,-0.218750,...,22.109375,2.171875,10.484375,1022.765625,2.031250,0.0,101.796875,2.953125,-24.776109,134.755000
1,new zealand,12.343750,13.656250,8.562500,5.625000,4.359375,3.015625,2.93750,3.609375,4.671875,...,11.296875,8.187500,11.265625,1025.468750,2.375000,0.0,166.953125,3.531250,-41.500083,172.834408
2,papua new guinea,13.250000,11.703125,7.125000,3.093750,1.203125,-0.203125,-2.68750,-3.687500,-3.906250,...,25.906250,2.250000,10.859375,1011.218750,2.125000,0.0,101.406250,3.609375,-5.681607,144.248908
3,south africa,13.359375,12.375000,11.875000,10.796875,10.250000,9.296875,8.34375,6.156250,5.359375,...,17.640625,2.609375,12.609375,1016.890625,4.718750,0.0,210.000000,3.343750,-28.816624,24.991639
4,usa,11.046875,12.171875,11.328125,10.046875,9.437500,8.968750,7.96875,6.234375,4.937500,...,23.671875,-0.250000,8.234375,1014.187500,3.140625,0.0,119.921875,2.281250,39.783730,-100.445882


In [276]:
df_final = pd.merge(left = df_attacks_paises, right= df_meteo_medias, how= "left", left_on= "country", right_on= "pais")
df_final.head(2)

,case_number,mes,country,type,area,activity,sex,species,injury,fatal,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,1791.00.00,NaN,australia,Unprovoked,New South Wales,NaN,F,NaN,"FATAL, ""bitten in two""",y,...,22.109375,2.171875,10.484375,1022.765625,2.03125,0.0,101.796875,2.953125,-24.776109,134.755
1,1788.05.10,May,australia,Boat,New South Wales,Fishing,M,NaN,"No injury to occupants, shark bit oar and rudder",n,...,22.109375,2.171875,10.484375,1022.765625,2.03125,0.0,101.796875,2.953125,-24.776109,134.755


In [277]:
list(df_final.columns)

['case_number',
 'mes',
 'country',
 'type',
 'area',
 'activity',
 'sex',
 'species',
 'injury',
 'fatal',
 'year',
 'age',
 'pais',
 'rh_950mb',
 'rh_900mb',
 'rh_850mb',
 'rh_800mb',
 'rh_750mb',
 'rh_700mb',
 'rh_650mb',
 'rh_600mb',
 'rh_550mb',
 'rh_500mb',
 'rh_450mb',
 'rh_400mb',
 'rh_350mb',
 'rh_300mb',
 'rh_250mb',
 'rh_200mb',
 'windspeed_950mb',
 'windspeed_900mb',
 'windspeed_850mb',
 'windspeed_800mb',
 'windspeed_750mb',
 'windspeed_700mb',
 'windspeed_650mb',
 'windspeed_600mb',
 'windspeed_550mb',
 'windspeed_500mb',
 'windspeed_450mb',
 'windspeed_400mb',
 'windspeed_350mb',
 'windspeed_300mb',
 'windspeed_250mb',
 'windspeed_200mb',
 'timepoint',
 'cloudcover',
 'highcloud',
 'midcloud',
 'lowcloud',
 'temp2m',
 'lifted_index',
 'rh2m',
 'msl_pressure',
 'prec_amount',
 'snow_depth',
 'wind10m.direction',
 'wind10m.speed',
 'latitud',
 'longitud']

In [278]:
df_final.to_csv('../../datos/attacks_meteo.csv')